## Step 1: Import PuLP modeler functions

In [1]:
from pulp import *
import pandas as pd

## Step 2: Define a variable using LpProblem to represent the optimization problem

- give the variable a name, for examle **probA**
- use **LpProblem** class: **LpProblem(name='NoName', sense=1)**. Parameters are explained below:
    - **name**:name of the problem used in the output .lp file
    - **sense**: type of the LP problem objective. Either LpMinimize (default) or LpMaximize.
- **LpProblem** returns an LP problem

In [2]:
probA=LpProblem("Problem A",LpMinimize)

## Step 3: Define decision variables using LpVariable.dicts
- give each decision variable a name, say **xhc** represents the number passengers to fly from Houston to Chicago
- use **LpVariable.dicts(name,index, lowBound=None, upBound=None, cat='Continuous')** to create a dictionary of LP variables Parameters are explained below:
    - **name**: The prefix to the name of each LP variable created
    - **indexs**:A list of strings of the keys to the dictionary of LP variables, and the main part of the variable name itself
    - **lowBound**: The lower bound on this variable’s range. Default is negative infinity
    - **upBound**: The upper bound on this variable’s range. Default is positive infinity
    - **cat**:  The category this variable is in, Integer, Binary or Continuous(default)

In [3]:
#Constraint and Decision Variable
Starting_month = ["Month1","Month2","Month3","Month4","Month5"]
Length_of_lease = ["1","2","3","4","5"]
Demand = [25,10,20,10,5]
Demand = dict(zip(Starting_month,Demand))
Cost = [300,525,775,850,975]
Cost = makeDict([Length_of_lease],Cost)

#Define decision variables with the starting month and lease length. 
#Starting from month 1, we have leases with 5 length. From month 2, we have 4. ... From month 5, we have only 1. 
for i in range(len(Starting_month)):
    if i==0:
        Space_lease_vars = LpVariable.dicts("x",(Starting_month[i:i+1],Length_of_lease[0:5-i]),lowBound=0,cat='Continuous')
    else:
        Space_lease_vars.update(LpVariable.dicts("x",(Starting_month[i:i+1],Length_of_lease[0:5-i]),lowBound=0,cat='Continuous'))

Space_lease_vars

{'Month1': {'1': x_Month1_1,
  '2': x_Month1_2,
  '3': x_Month1_3,
  '4': x_Month1_4,
  '5': x_Month1_5},
 'Month2': {'1': x_Month2_1,
  '2': x_Month2_2,
  '3': x_Month2_3,
  '4': x_Month2_4},
 'Month3': {'1': x_Month3_1, '2': x_Month3_2, '3': x_Month3_3},
 'Month4': {'1': x_Month4_1, '2': x_Month4_2},
 'Month5': {'1': x_Month5_1}}

## Step 4: Define objective and constraints using += operator and lpSum
- **lpSum(vector)** calculates the sum of a list of linear expressions. 
- The parameter is a list of linear expressions

- **Constraints**: (Cover space requirement of each month)
- x_month1_1 + x_month1_2 + x_month1_3 + x_month1_4 + x_month1_5 >= 25
- x_month1_2 + x_month1_3 + x_month1_4 + x_month1_5 + x_month2_1 + x_month2_2 + x_month2_3 + x_month2_4 >= 10
- x_month1_3 + x_month1_4 + x_month1_5 + x_month2_2 + x_month2_3+ x_month2_4 + x_month3_1 + x_month3_2 + x_month3_3 >= 20
- x_month1_4 + x_month1_5 + x_month2_3 + x_month2_4 + x_month3_2 + x_month3_3 + x_month4_1 + x_month4_2 >= 10
- x_month1_5 + x_month2_4 + x_month3_3 + x_month4_2 + x_month5_1 >= 5

<img src="Picture1.png">

In [4]:
#Objective Function

probA+=lpSum(Space_lease_vars[Starting_month[i]][j]*Cost[j] for i in range(len(Starting_month)) for j in Length_of_lease[0:5-i])
#Minimize the total cost of all the leases. 

#Demand constraint
for i in range(len(Starting_month)):
    probA += lpSum(Space_lease_vars[Starting_month[j]][Length_of_lease[k]] for j in range(i+1) for k in range(i-j,5-j) ) >= Demand[Starting_month[i]]
# range(i+1) describes how many months we need to consider in each starting month (all the months before). 
# range (i-j,5-j) describes what are the leases we need to look at.
#For each starting month i, we keep all the lease that start from this month and other leases that cover this month. 
#For example, in month 1, we keep all the lease starting from month 1. Which is: x_month1_1 + x_month1_2 + x_month1_3 + x_month1_4 + x_month1_5
#In month 2, we keep all the lease starting from month 2 and leases that start from month 1 but with length larger or equal than 2. 
#Which is: x_month1_2 + x_month1_3 + x_month1_4 + x_month1_5 + x_month2_1 + x_month2_2 + x_month2_3 + x_month2_4
#Month 3: x_month1_3 + x_month1_4 + x_month1_5 + x_month2_2 + x_month2_3 + x_month2_4 + x_month3_1 + x_month3_2 + x_month3_3


In [5]:
probA

Problem A:
MINIMIZE
300*x_Month1_1 + 525*x_Month1_2 + 775*x_Month1_3 + 850*x_Month1_4 + 975*x_Month1_5 + 300*x_Month2_1 + 525*x_Month2_2 + 775*x_Month2_3 + 850*x_Month2_4 + 300*x_Month3_1 + 525*x_Month3_2 + 775*x_Month3_3 + 300*x_Month4_1 + 525*x_Month4_2 + 300*x_Month5_1 + 0
SUBJECT TO
_C1: x_Month1_1 + x_Month1_2 + x_Month1_3 + x_Month1_4 + x_Month1_5 >= 25

_C2: x_Month1_2 + x_Month1_3 + x_Month1_4 + x_Month1_5 + x_Month2_1
 + x_Month2_2 + x_Month2_3 + x_Month2_4 >= 10

_C3: x_Month1_3 + x_Month1_4 + x_Month1_5 + x_Month2_2 + x_Month2_3
 + x_Month2_4 + x_Month3_1 + x_Month3_2 + x_Month3_3 >= 20

_C4: x_Month1_4 + x_Month1_5 + x_Month2_3 + x_Month2_4 + x_Month3_2
 + x_Month3_3 + x_Month4_1 + x_Month4_2 >= 10

_C5: x_Month1_5 + x_Month2_4 + x_Month3_3 + x_Month4_2 + x_Month5_1 >= 5

VARIABLES
x_Month1_1 Continuous
x_Month1_2 Continuous
x_Month1_3 Continuous
x_Month1_4 Continuous
x_Month1_5 Continuous
x_Month2_1 Continuous
x_Month2_2 Continuous
x_Month2_3 Continuous
x_Month2_4 Continuo

## Step 5: Run solver

In [6]:
#probA.writeLP("Space_lease.lp") #optional
probA.solve()
print("Status:",LpStatus[probA.status])

Status: Optimal


## Step 6: Format PuLP Solution Output

In [7]:
print("Total cost=", value(probA.objective))
output=[]

for i in range(len(Starting_month)):
    var_output=[]
    for k in Length_of_lease[0:5-i]:
        var_output.append(Space_lease_vars[Starting_month[i]][k].varValue)
        
    output.append(var_output)
    
print(output)
output_df = pd.DataFrame(output,index=Starting_month,columns=Length_of_lease)
output_df

Total cost= 16625.0
[[15.0, 0.0, 0.0, 5.0, 5.0], [0.0, 0.0, 0.0, 0.0], [10.0, 0.0, 0.0], [0.0, 0.0], [0.0]]


,1,2,3,4,5
Month1,15.0,0.0,0.0,5.0,5.0
Month2,0.0,0.0,0.0,0.0,NaN
Month3,10.0,0.0,0.0,NaN,NaN
Month4,0.0,0.0,NaN,NaN,NaN
Month5,0.0,NaN,NaN,NaN,NaN
